In [27]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import os
from gensim import corpora

In [2]:
with open('languages.txt', 'r') as file:
    languages = [f.strip('\n') for f in file.readlines()]

## CORPUSES

#### read common articles in all languages

In [3]:
def read_text(datatype, folder, filename):
    with open(os.path.join(datatype, folder, filename), 'r') as file:
        return file.read()

In [4]:
files = {lan: os.listdir(os.path.join('page_text', lan)) for lan in languages}

In [5]:
for i, key in enumerate(files.keys()):
    if i == 0:
        common_files = set(files[key])
    else:
        common_files = common_files.intersection(set(files[key]))

In [33]:
# plain texts
texts = {lan: [read_text('page_text', lan, f).lower() 
               for f in os.listdir(os.path.join('page_text', lan)) if f in common_files] 
         for lan in languages}

In [35]:
# tokenized text - remove punctuation
tokenizer = RegexpTokenizer(r'\w+')
texts_split = {lan: [tokenizer.tokenize(text) for text in texts[lan]] for lan in languages}

In [37]:
text_dicts = {lan: corpora.Dictionary(texts_split[lan]) for lan in languages}

In [13]:
def get_len(unit, text_dict):
    if unit == 'char':
        return {key: len(' '.join(value)) for key, value in text_dict.items()}
    elif unit == 'word':
        ret = {}
        for key, value in text_dict.items():
            val = []
            for f in value:
                val += f
            print(f"{key}: {len(val)}")

In [16]:
get_len('word', texts_split)

en: 708315
de: 483394
hu: 201883
ro: 201783


In [ ]:
print('Length in characters:')
for key, value in texts_notsplit.items():
    print(f"{key}: {len(' '.join(value))}")

In [ ]:
print('Length in words:')
for key, value in texts.items():
    val = []
    for f in value:
        val += f
    print(f"{key}: {len(val)}")

In [ ]:
print('Length of unique words:')
for key in languages:
    print(f"{key}: {len(text_dicts[key])}")

In [ ]:
languages_long = {'en': 'english', 'de': 'german', 'hu': 'hungarian', 'ro': 'romanian'}

In [ ]:
stopwords_bylang = {lan: set(stopwords.words(languages_long[lan])) for lan in languages}

In [ ]:
texts_stops = {lan: [[g for g in f if not g in stopwords_bylang[lan]] for f in texts[lan]] for lan in languages}

In [ ]:
text_stop_dicts = {lan: corpora.Dictionary(texts_stops[lan]) for lan in languages}

In [ ]:
print('Length in words without stops:')
for key, value in texts_stops.items():
    val = []
    for f in value:
        val += f
    print(f"{key}: {len(val)}")

In [ ]:
print('Length of unique words:')
for key in languages:
    print(f"{key}: {len(text_stop_dicts[key])}")